### SageMaker Payment Classification 
#### *Classify financial transactions*

#### Background <a class="anchor" id="Background"></a>

This notebook demonstrates how you can train and deploy a machine learning to classify payment transactions. Enriching financial transactions with the category of the transaction. This can be used as an intermediate step in fraud detection, personalization or anomaly detection. As well as a method to provide end users (e.g. customers at a bank) with an overview of their spending habits. Amazon SageMaker can be used to train and deploy a XGBoost model, as well as the required underlying infrastructure. For this notebook a generated dataset is used where a payment consists of mostly an amount, sender, receiver and timestamp.


#### Notebook overview <a class="anchor" id="Notebook-overview"></a>

This notebook consists of seven parts. First, we import and configure the required libraries. After that we prepare the data used in this example and create the feature store. With the newly created features we create a XGBoost model. An endpoint is created to host this model. We evaluate the performance of the model and end by cleaning up the used resources.

#### Dataset <a class="anchor" id="Dataset"></a>

For this notebook we use a synthetic dataset. This dataset has the following features 

* __transaction_category__: The category of the transaction, this is one of the next 19 options.

               'Uncategorized', 'Entertainment', 'Education',
                    'Shopping', 'Personal Care', 'Health and Fitness',
             'Food and Dining', 'Gifts and Donations', 'Investments',
         'Bills and Utilities', 'Auto and Transport', 'Travel',
            'Fees and Charges', 'Business Services', 'Personal Services',
                       'Taxes', 'Gambling', 'Home',
      'Pension and insurances'


* __receiver_id__: an identifier for the receiving party. The identifier consist of 16 numbers.
* __sender_id__: an identifier for the sending party. The identifier consist of 16 numbers.
* __amount__: the amount which is transferred.
* __timestamp__: the timestamp of the transaction in YYYY-MM-DD HH:MM:SS format.
____

#### 1. Setup <a class="anchor" id="Setup"></a>

Before we start we need to update the sagemaker library

In [1]:
import sys

!{sys.executable} -m pip install --upgrade pip             --quiet # upgrade pip to the latest vesion
!{sys.executable} -m pip install --upgrade sagemaker==2.64 --quiet # upgrade SageMaker to the latest vesion

Now that we have the latest version we can import the libraries that we'll use in this notebook

In [2]:
import boto3
import io
import sagemaker
import time
import os

from time import sleep
from sklearn.metrics import classification_report
from sagemaker.feature_store.feature_group import FeatureGroup

import pandas as pd
import numpy as np

Let's set the session variables to ensure that SageMaker is configured correctly.

In [3]:
region = sagemaker.Session().boto_region_name
sm_client = boto3.client("sagemaker")
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session, sagemaker_client=sm_client)
role = sagemaker.get_execution_role()
bucket_prefix = "SageMaker_Payment_Classification"
s3_bucket = sagemaker_session.default_bucket()

We define the factorize key which is used to map the '__transaction_category__' to numeric values

In [4]:
factorize_key = {
    "Uncategorized": 0,
    "Entertainment": 1,
    "Education": 2,
    "Shopping": 3,
    "Personal Care": 4,
    "Health and Fitness": 5,
    "Food and Dining": 6,
    "Gifts and Donations": 7,
    "Investments": 8,
    "Bills and Utilities": 9,
    "Auto and Transport": 10,
    "Travel": 11,
    "Fees and Charges": 12,
    "Business Services": 13,
    "Personal Services": 14,
    "Taxes": 15,
    "Gambling": 16,
    "Home": 17,
    "Pension and insurances": 18,
}

____
#### 2. Data preparation <a class="anchor" id="Data-preparation"></a>

Let's start by loading the dataset from our csv file into a Pandas DataFrame

In [5]:
data = pd.read_csv(
    "financial_transactions_mini.csv",
    parse_dates=["timestamp"],
    infer_datetime_format=True,
    dtype={"transaction_category": "string"},
)

The dataframe looks as follows:

In [6]:
data.sample(10)

,transaction_category,receiver_id,sender_id,amount,timestamp
88228,Auto and Transport,4032086248573357,4854198561438962,40.52,2021-02-02 16:53:53
53031,Health and Fitness,4004008505897877,4129090439874560,20.45,2021-01-12 12:42:26
6709,Entertainment,4498329552166443,4558676880154880,28.06,2021-01-12 23:43:22
30931,Shopping,4486974585482061,4346839998930966,231.21,2021-02-27 17:31:26
75241,Auto and Transport,4836715383646960,4014541554769975,114.66,2021-01-18 13:22:08
73496,Auto and Transport,4662781317435275,4385857335607719,98.44,2021-04-12 13:25:17
33725,Shopping,4991749551406205,4159891477673167,62.73,2021-03-16 18:48:50
77781,Auto and Transport,4294503963171905,4037516083070395,99.08,2021-01-29 15:16:34
15232,Education,4952268365995102,4035597918199411,1035.21,2021-03-29 07:30:02
87997,Auto and Transport,4108493722083201,4646563712289148,46.07,2021-04-25 12:42:00


Next, we extract the year, month, day, hour, minute, second from the timestamp and remove the timestamp

In [7]:
data["year"] = data["timestamp"].dt.year
data["month"] = data["timestamp"].dt.month
data["day"] = data["timestamp"].dt.day
data["hour"] = data["timestamp"].dt.hour
data["minute"] = data["timestamp"].dt.minute
data["second"] = data["timestamp"].dt.second

del data["timestamp"]

We'll transform the transaction categories to numeric targets for the classification by factorization.

In [8]:
data["transaction_category"] = data["transaction_category"].replace(factorize_key)

____
#### 3. Create feature store <a class="anchor" id="Create-feature-store"></a>

To enrich dataset we will use the [Feature Store](https://aws.amazon.com/sagemaker/feature-store/). 

Before creating the feature store itself we need to set a name for the feature group and identifier used

In [9]:
feature_group_name = "feature-group-payment-classification"
record_identifier_feature_name = "identifier"

With the name we defined we create the feature group, runtime and session

In [10]:
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sm_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

Once we have defined our feature store we need to put some data in it. We create a Pandas DataFrame with the columns mean_amount, count, identifier and EventTime to store in the feature store

In [11]:
columns = ["mean_amount", "count", "identifier", "EventTime"]
feature_store_data = pd.DataFrame(columns=columns)

feature_store_data["identifier"] = range(19)
feature_store_data["mean_amount"] = 0.0
feature_store_data["count"] = 1
feature_store_data["EventTime"] = time.time()

Using the created DataFrame we set the feature definitions

In [12]:
feature_group.load_feature_definitions(data_frame=feature_store_data)

[FeatureDefinition(feature_name='mean_amount', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='count', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='identifier', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='EventTime', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>)]

With these definitions ready we can create the feature group itself

In [13]:
feature_group.create(
    s3_uri=f"s3://{s3_bucket}/{bucket_prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:eu-west-1:125791339390:feature-group/feature-group-payment-classification',
 'ResponseMetadata': {'RequestId': '88ce557d-c89e-4b81-9668-5c37702da175',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '88ce557d-c89e-4b81-9668-5c37702da175',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Wed, 03 Nov 2021 12:12:26 GMT'},
  'RetryAttempts': 0}}

It takes a while for the feature group to be created, we need to wait for this to be done before trying to ingest data in the feature store

In [14]:
status = feature_group.describe().get("FeatureGroupStatus")
while status == "Creating":
    print("Waiting for Feature Group to be Created")
    time.sleep(5)
    status = feature_group.describe().get("FeatureGroupStatus")
print(f"FeatureGroup {feature_group.name} successfully created.")

Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
FeatureGroup feature-group-payment-classification successfully created.


Once the feature group is created we can ingest data into it

In [15]:
feature_group.ingest(data_frame=feature_store_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='feature-group-payment-classification', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f5aaee4e790>, max_workers=3, max_processes=1, _async_result=<multiprocess.pool.MapResult object at 0x7f5aae43ffd0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

To retrieve data from our feature store we define a function that gets the current values from the feature store

In [16]:
def get_feature_store_values():
    response = featurestore_runtime.batch_get_record(
        Identifiers=[
            {
                "FeatureGroupName": feature_group_name,
                "RecordIdentifiersValueAsString": [str(i) for i in range(19)],
            }
        ]
    )

    columns = ["mean_amount", "count", "identifier", "EventTime"]

    feature_store_resp = pd.DataFrame(
        data=[
            [resp["Record"][i]["ValueAsString"] for i in range(len(columns))]
            for resp in response["Records"]
        ],
        columns=columns,
    )
    feature_store_resp["identifier"] = feature_store_resp["identifier"].astype(int)
    feature_store_resp["count"] = feature_store_resp["count"].astype(int)
    feature_store_resp["mean_amount"] = feature_store_resp["mean_amount"].astype(float)
    feature_store_resp["EventTime"] = feature_store_resp["EventTime"].astype(float)
    feature_store_resp = feature_store_resp.sort_values(by="identifier")

    return feature_store_resp


feature_store_resp = get_feature_store_values()

We update the values in the feature store with the real values of our data

In [17]:
feature_store_data = pd.DataFrame()
feature_store_data["mean_amount"] = data.groupby(["transaction_category"]).mean()["amount"]
feature_store_data["count"] = data.groupby(["transaction_category"]).count()["amount"]
feature_store_data["identifier"] = feature_store_data.index
feature_store_data["EventTime"] = time.time()

feature_store_data["mean_amount"] = (
    pd.concat([feature_store_resp, feature_store_data])
    .groupby("identifier")
    .apply(lambda x: np.average(x["mean_amount"], weights=x["count"]))
)
feature_store_data["count"] = (
    pd.concat([feature_store_resp, feature_store_data]).groupby("identifier").sum()["count"]
)

feature_group.ingest(data_frame=feature_store_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='feature-group-payment-classification', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f5aaee4e790>, max_workers=3, max_processes=1, _async_result=<multiprocess.pool.MapResult object at 0x7f5aae448a10>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

And display them after getting them from the feature store

In [18]:
feature_store_data = get_feature_store_values()
feature_store_data

,mean_amount,count,identifier,EventTime
2,494.773326,466,0,1.635942e+09
15,51.205853,14513,1,1.635942e+09
11,850.011007,745,2,1.635942e+09
14,100.914531,33954,3,1.635942e+09
5,31.478000,1210,4,1.635942e+09
8,119.571685,4838,5,1.635942e+09
4,93.239755,9675,6,1.635942e+09
16,51.054155,2792,7,1.635942e+09
17,6018.076434,931,8,1.635942e+09
10,114.745296,3350,9,1.635942e+09


We use the feature store to calculate the distance between the average of every category and the current amount

In [19]:
additional_features = pd.pivot_table(
    feature_store_data, values=["mean_amount"], index=["identifier"]
).T.add_suffix("_dist")
additional_features_columns = list(additional_features.columns)
data = pd.concat([data, pd.DataFrame(columns=additional_features_columns)])
data[additional_features_columns] = additional_features.values[0]
for col in additional_features_columns:
    data[col] = abs(data[col] - data["amount"])

data

,transaction_category,receiver_id,sender_id,amount,year,month,day,hour,minute,second,...,9_dist,10_dist,11_dist,12_dist,13_dist,14_dist,15_dist,16_dist,17_dist,18_dist
0,0.0,4.518552e+15,4.333582e+15,833.26,2021.0,3.0,10.0,19.0,57.0,42.0,...,718.514704,732.288109,481.72241,806.338389,628.667464,337.027497,2074.16073,460.037701,52.287679,627.986786
1,0.0,4.518552e+15,4.642413e+15,596.63,2021.0,2.0,11.0,17.0,53.0,32.0,...,481.884704,495.658109,245.09241,569.708389,392.037464,100.397497,2310.79073,223.407701,184.342321,391.356786
2,0.0,4.274544e+15,4.952666e+15,176.76,2021.0,2.0,21.0,18.0,29.0,32.0,...,62.014704,75.788109,174.77759,149.838389,27.832536,319.472503,2730.66073,196.462299,604.212321,28.513214
3,0.0,4.518552e+15,4.457299e+15,879.78,2021.0,4.0,9.0,16.0,14.0,19.0,...,765.034704,778.808109,528.24241,852.858389,675.187464,383.547497,2027.64073,506.557701,98.807679,674.506786
4,0.0,4.601853e+15,4.578126e+15,742.25,2021.0,4.0,4.0,15.0,50.0,16.0,...,627.504704,641.278109,390.71241,715.328389,537.657464,246.017497,2165.17073,369.027701,38.722321,536.976786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,18.0,4.405008e+15,4.583356e+15,205.43,2021.0,4.0,20.0,12.0,23.0,53.0,...,90.684704,104.458109,146.10759,178.508389,0.837464,290.802503,2701.99073,167.792299,575.542321,0.156786
99993,18.0,4.300417e+15,4.949241e+15,151.49,2021.0,3.0,24.0,19.0,30.0,18.0,...,36.744704,50.518109,200.04759,124.568389,53.102536,344.742503,2755.93073,221.732299,629.482321,53.783214
99994,18.0,4.405008e+15,4.996896e+15,188.28,2021.0,3.0,8.0,19.0,51.0,10.0,...,73.534704,87.308109,163.25759,161.358389,16.312536,307.952503,2719.14073,184.942299,592.692321,16.993214
99995,18.0,4.262047e+15,4.017367e+15,204.26,2021.0,2.0,14.0,23.0,25.0,7.0,...,89.514704,103.288109,147.27759,177.338389,0.332536,291.972503,2703.16073,168.962299,576.712321,1.013214


____
#### 4. Create model <a class="anchor" id="Create-model"></a>
In this notebook we will be using the [eXtreme Gradient Boosting](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) (XGBoost) implementation of the gradient boosted trees algorithm. This model is selected due to it relatively fast training time and explainable properties. The model can be substituted at will a different [SageMaker estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) or a [model of your choosing](https://aws.amazon.com/blogs/machine-learning/bring-your-own-model-with-amazon-sagemaker-script-mode/).



Now that we have the dataset we can start preparing the model. First, we create a training, validation and testing split.

In [20]:
# Randomly sort the data then split out first 70%, second 20%, and last 10%
train_data, validation_data, test_data = np.split(
    data.sample(frac=1, random_state=42), [int(0.7 * len(data)), int(0.9 * len(data))]
)

We save these sets to a file.

In [21]:
train_data.to_csv("train.csv", index=False, header=False)
validation_data.to_csv("validation.csv", index=False, header=False)
test_data.to_csv("test.csv", index=False, header=False)

And upload these files to our s3 bucket

In [22]:
boto3.Session().resource("s3").Bucket(s3_bucket).Object(
    os.path.join(bucket_prefix, "train/train.csv")
).upload_file("train.csv")
boto3.Session().resource("s3").Bucket(s3_bucket).Object(
    os.path.join(bucket_prefix, "validation/validation.csv")
).upload_file("validation.csv")

Get the xgboost sagemaker image

In [23]:
container = sagemaker.image_uris.retrieve(region=region, framework="xgboost", version="1.2-2")

Transform our data to a sagemaker input for training

In [24]:
s3_input_train = sagemaker.inputs.TrainingInput(
    s3_data="s3://{}/{}/train".format(s3_bucket, bucket_prefix), content_type="csv"
)
s3_input_validation = sagemaker.inputs.TrainingInput(
    s3_data="s3://{}/{}/validation/".format(s3_bucket, bucket_prefix), content_type="csv"
)

We define the XGBoost model

In [25]:
xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(s3_bucket, bucket_prefix),
    sagemaker_session=sagemaker_session,
)

Set the parameters

In [26]:
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective="multi:softprob",
    num_class=19,
    verbosity=0,
    num_round=100,
)

And train the model

In [27]:
xgb.fit({"train": s3_input_train, "validation": s3_input_validation})

2021-11-03 12:13:08 Starting - Starting the training job...
2021-11-03 12:13:31 Starting - Launching requested ML instancesProfilerReport-1635941588: InProgress
......
2021-11-03 12:14:31 Starting - Preparing the instances for training.........
2021-11-03 12:16:05 Downloading - Downloading input data...
2021-11-03 12:16:32 Training - Downloading the training image.....[2021-11-03 12:17:19.792 ip-10-0-112-240.eu-west-1.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-11-03:12:17:19:INFO] Imported framework sagemaker_xgboost_container.training
[2021-11-03:12:17:19:INFO] Failed to parse hyperparameter objective value multi:softprob to Json.
Returning the value itself
[2021-11-03:12:17:19:INFO] No GPUs detected (normal if no gpus installed)
[2021-11-03:12:17:19:INFO] Running XGBoost Sagemaker in algorithm mode
[2021-11-03:12:17:19:INFO] Determined delimiter of CSV input is ','
[2021-11-03:12:17:19:INFO] Determined delimiter of CSV input is ','
[2021-11-03:12:1

____
#### 5. Using the endpoint <a class="anchor" id="Using-the-endpoint"></a>

Deploy the model to an endpoint

In [28]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    serializer=sagemaker.serializers.CSVSerializer(),
)

-------!

____
#### 6. Evaluate performance <a class="anchor" id="Evaluate-performance"></a>

Run the model on our test data

In [29]:
def predict(data, predictor):
    predictions = []
    confidences = []
    for row in data:
        response = np.fromstring(predictor.predict(row).decode("utf-8")[1:], sep=",")
        pred = response.argmax()
        confidence = max(response)
        predictions.extend([pred])
        confidences.extend([confidence])

    return predictions, confidences

Running it on the first 3 rows in our dataset results in the following:

In [30]:
pred, conf = predict(test_data.drop(["transaction_category"], axis=1).to_numpy()[:3], xgb_predictor)
print(
    f"The predictions for the first 3 entries are {pred}, the confidence for these predictions are {conf}"
)

The predictions for the first 3 entries are [1, 1, 11], the confidence for these predictions are [0.5061870217323303, 0.5919070839881897, 0.658933162689209]


/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.


Now we run the predictions on the complete dataset

In [31]:
predictions, confidences = predict(
    test_data.drop(["transaction_category"], axis=1).to_numpy(), xgb_predictor
)

/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.


And report the prediction results

In [32]:
print(
    classification_report(
        test_data["transaction_category"].to_list(), predictions, target_names=factorize_key
    )
)

                        precision    recall  f1-score   support

         Uncategorized       1.00      0.92      0.96        51
         Entertainment       0.81      0.89      0.85      1486
             Education       1.00      0.94      0.97        80
              Shopping       0.86      0.94      0.90      3441
         Personal Care       1.00      0.98      0.99       132
    Health and Fitness       0.99      0.89      0.94       443
       Food and Dining       0.99      0.82      0.90       918
   Gifts and Donations       1.00      0.95      0.97       275
           Investments       0.99      0.97      0.98        88
   Bills and Utilities       1.00      0.99      1.00       332
    Auto and Transport       0.94      0.84      0.88      1967
                Travel       0.96      0.84      0.90       120
      Fees and Charges       1.00      0.94      0.97       106
     Business Services       1.00      0.99      1.00       146
     Personal Services       1.00      

____
#### 7. Clean up <a class="anchor" id="Clean-up"></a>

Remove the feature group and endpoint to clean up

In [33]:
feature_group.delete()
xgb_predictor.delete_endpoint(delete_endpoint_config=True)